# 1 Thêm thư viện cần thiết

In [27]:
import torch.optim as optim
from src.train import train_epoch, evaluate, epoch_time
import time
import torch.nn as nn
import math
import itertools
import sys
import os
import torch

In [ ]:
# --- CHÈN ĐOẠN NÀY VÀO GIỮA CELL IMPORT VÀ CELL CẤU HÌNH ---

# Thêm đường dẫn để import được code từ thư mục src
sys.path.append(os.path.abspath('..'))

from src.dataset import get_loaders
from src.model import Encoder, Decoder, Seq2Seq

# 1. Cấu hình thiết bị
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔹 Đang sử dụng thiết bị: {device}")

# 2. Load Dữ liệu (Để có train_loader, vocab_en, vocab_fr)
BATCH_SIZE = 64
path_en = '../data/raw/train.en'
path_fr = '../data/raw/train.fr'

print("⏳ Đang tải dữ liệu...")
full_train_loader, vocab_en, vocab_fr = get_loaders(BATCH_SIZE, path_en, path_fr)
#train_loader = list(itertools.islice(full_train_loader, 5))
train_loader=full_train_loader
# 3. Khởi tạo Mô hình (Để có biến model)
INPUT_DIM = len(vocab_en)
OUTPUT_DIM = len(vocab_fr)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DROPOUT)
model = Seq2Seq(enc, dec, device).to(device)

# Khởi tạo trọng số (giúp model học nhanh hơn)
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            torch.nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            torch.nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

print("✅ Đã khởi tạo xong Data và Model. Sẵn sàng cấu hình huấn luyện!")

🔹 Đang sử dụng thiết bị: cpu
⏳ Đang tải dữ liệu...
⏳ Building Vocab...
✅ Đã khởi tạo xong Data và Model. Sẵn sàng cấu hình huấn luyện!


# 2 Cấu hình huấn luyện

In [29]:
# Cấu hình theo gợi ý của đề bài
LEARNING_RATE = 0.001
N_EPOCHS = 10       # Chạy thử 10 vòng
CLIP = 1.0          # Cắt gradient nếu lớn hơn 1
PATIENCE = 3        # Early Stopping: Dừng nếu sau 3 lần không tiến bộ

# Tạo thư mục lưu model nếu chưa có
import os
if not os.path.exists('../checkpoints'):
    os.makedirs('../checkpoints')

model_save_path = '../checkpoints/best_model.pth'

# Bỏ qua việc tính loss cho token <pad>
PAD_IDX = vocab_fr['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

# Dùng Adam Optimizer
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 3 Tạo vòng lặp huấn luyện

In [30]:
print(f"Bắt đầu huấn luyện: {device}")
best_valid_loss = float('inf')
no_improve_count = 0  # Đếm số lần không tiến bộ

train_losses = []
valid_losses = []

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    # 1. Train
    train_loss = train_epoch(model, train_loader, optimizer, criterion, CLIP, device)
    
    # 2. Evaluate (Dùng tập train tạm nếu chưa có val_loader, nhưng đúng ra phải dùng val_loader)
    # Lưu ý: Bạn cần tạo thêm val_loader tương tự train_loader ở bước trước
    valid_loss = evaluate(model, train_loader, criterion, device) 
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # Lưu lại lịch sử để vẽ biểu đồ
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    # --- EARLY STOPPING & CHECKPOINT ---
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), model_save_path) # Lưu model tốt nhất
        print(f"Đã lưu model tốt hơn tại epoch {epoch+1}!")
        no_improve_count = 0
    else:
        no_improve_count += 1
        print(f"Loss không giảm ({no_improve_count}/{PATIENCE})")
        
    if no_improve_count >= PATIENCE:
        print("Dừng sớm (Early Stopping) vì model không còn học tốt hơn nữa.")
        break
        
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Bắt đầu huấn luyện: cpu
Đã lưu model tốt hơn tại epoch 1!
Epoch: 01 | Time: 0m 15s
	Train Loss: 9.198 | Train PPL: 9877.793
	 Val. Loss: 9.085 |  Val. PPL: 8819.658
Đã lưu model tốt hơn tại epoch 2!
Epoch: 02 | Time: 0m 15s
	Train Loss: 7.564 | Train PPL: 1927.334
	 Val. Loss: 5.466 |  Val. PPL: 236.575
Đã lưu model tốt hơn tại epoch 3!
Epoch: 03 | Time: 0m 15s
	Train Loss: 5.336 | Train PPL: 207.611
	 Val. Loss: 5.166 |  Val. PPL: 175.133
Đã lưu model tốt hơn tại epoch 4!
Epoch: 04 | Time: 0m 15s
	Train Loss: 5.137 | Train PPL: 170.254
	 Val. Loss: 5.027 |  Val. PPL: 152.404
Đã lưu model tốt hơn tại epoch 5!
Epoch: 05 | Time: 0m 15s
	Train Loss: 5.014 | Train PPL: 150.501
	 Val. Loss: 4.928 |  Val. PPL: 138.085
Đã lưu model tốt hơn tại epoch 6!
Epoch: 06 | Time: 0m 15s
	Train Loss: 4.918 | Train PPL: 136.722
	 Val. Loss: 4.848 |  Val. PPL: 127.433
Đã lưu model tốt hơn tại epoch 7!
Epoch: 07 | Time: 0m 16s
	Train Loss: 4.845 | Train PPL: 127.115
	 Val. Loss: 4.787 |  Val. PPL: 119.950
